In [5]:
#| default_exp routes.group

In [6]:
#| exporti
import httpx
#from .get_data import get_data
#from ..DomoAuth import DomoFullAuth
#from ...utils.ResponseGetData import ResponseGetData

import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [11]:
# | export
def generate_body_create_group(group_name: str,
                               group_type: str = 'open',
                               description: str = '') -> dict:
    """ Generates the body to create group for v2 API"""
    body = {"name": group_name, 
            "type": group_type,
            "description": description}

    return body


In [14]:
#| export
async def create_group(auth: dmda.DomoAuth,
                       group_name: str,
                       group_type: str = 'open',
                       description: str = '',
                       debug_api: bool = False) -> rgd.ResponseGetData:
    # body : {"name": "GROUP_NAME", "type": "open", "description": ""}

    body = generate_body_create_group(
        group_name=group_name, group_type=group_type, description=description)

    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/'

    res= await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        body=body,
        debug_api=debug_api
    )

    if not res.is_success:
        raise Error

#### Sample implementation of create_group

In [24]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

token = await token_auth.get_auth_token()
headers = token_auth.generate_auth_header(token = token)

# res = await create_group(auth=token_auth, group_name='Test Group 2', debug_api=True)
# res.response

body = generate_body_create_group(
        group_name="test group4", group_type="open", description="")

url = f'https://{token_auth.domo_instance}.domo.com/api/content/v2/groups/'

import httpx

session = httpx.AsyncClient()


await session.request(method = 'POST', url = url, headers =headers , json=body )

<Response [400 Bad Request]>

In [10]:
#| export
async def get_all_groups(full_auth: DomoFullAuth,
                         log_results: bool = False,
                         debug: bool = False,
                         session: httpx.AsyncClient = None) -> rgd:
    if debug:
        print(full_auth)

    url = f'https://{full_auth.domo_instance}.domo.com/api/content/v2/groups/grouplist'

    if debug:
        print(full_auth, url)

    if log_results:
        print(f'Getting groups from - {url}')

    res = await gd(url=url, method='GET', auth=full_auth, session=session)

    return res

NameError: name 'DomoFullAuth' is not defined

In [ ]:
#| export
async def search_groups_by_name(full_auth: DomoFullAuth,
                                search_name: str,
                                debug: bool = False, log_results: bool = False) -> rgd:
    url = f'https://{full_auth.domo_instance}.domo.com/api/content/v2/groups/grouplist?ascending=true&search={search_name}&sort=name '

    res = await gd(
        auth=full_auth,
        url=url,
        method='GET',
        debug=debug,
        log_results=log_results
    )
    return res

In [ ]:
#| export
def generate_body_update_group_membership(group_id: str,
                                          add_user_arr: list[str] = None,
                                          remove_user_arr: list[str] = None,
                                          add_owner_user_arr: list[str] = None,
                                          remove_owner_user_arr: list[str] = None) -> list[dict]:
    body = {"groupId": int(group_id)}
    if add_owner_user_arr:
        body.update({"addOwners": [{"type": "USER", "id": str(
            userId)} for userId in add_owner_user_arr]})

    if remove_owner_user_arr:
        body.update({"removeOwners": [{"type": "USER", "id": str(
            userId)} for userId in remove_owner_user_arr]})

    if remove_user_arr:
        body.update({"removeMembers": [
                    {"type": "USER", "id": str(userId)} for userId in remove_user_arr]})
    if add_user_arr:
        body.update(
            {"addMembers": [{"type": "USER", "id": str(userId)} for userId in add_user_arr]})

    return [body]

In [ ]:
#| export
async def update_group_membership(full_auth: DomoFullAuth,
                                  body: dict,
                                  log_results: bool = False, debug: bool = False) -> ResponseGetData:
    # body = [{
    #     "groupId":"GROUP_ID",
    #     "removeMembers": [{"type":"USER","id":"USER_ID"}],
    #     "addMembers"   : [{"type":"USER","id":"USER_ID"}]
    # }]

    url = f'https://{full_auth.domo_instance}.domo.com/api/content/v2/groups/access'

    if debug:
        print(url, body)

    res = await gd(
        auth=full_auth,
        url=url,
        method='PUT',
        body=body,
        debug=debug
    )

    return res

In [ ]:
#| export
async def get_group_by_id(full_auth:DomoFullAuth, group_id:id, debug:bool = False) -> ResponseGetData:
    url = f'https://{full_auth.domo_instance}.domo.com/api/content/v2/groups/{group_id}'

    if debug:
        print(url)

    res = await gd(
        auth=full_auth,
        url=url,
        method='GET',
        debug=debug
    )

    return res